In [1067]:
import numpy as np
import pandas as pd
from datetime import datetime
import time

In [1068]:
bowery = pd.read_csv('bowery/bowery.csv')

In [1069]:
import re

# strip white space from columns
bowery.location = bowery.location.str.strip()
bowery.date = bowery.date.str.strip()
bowery.headline_artist_name = bowery.headline_artist_name.str.strip()
bowery.other_artists_name = bowery.other_artists_name.str.strip()
bowery.price = bowery.price.str.strip()
bowery.time = bowery.time.str.strip()


In [1070]:
# split state and city, drop location
bowery['city'], bowery['state'] = bowery['location'].str.split(',', 1).str
bowery = bowery.drop(['location'], 1)

# split day of week and date, drop date
bowery['week_day'], bowery['date'] = bowery['date'].str.split(',', 1).str

# split doors time and show time, drop time
bowery['doors_time'], bowery['show_time'] = bowery['time'].str.split(',', 1).str
bowery = bowery.drop(['time'], 1)


In [1071]:
# clean door_time and show_time

def time(x):
    clean_time = []
    for i in x:
        try:
            m = re.search("\d+:\d+ [PA][M]", i)
            clean_time.append(m.group(0))
        except:
            clean_time.append(None)
    return clean_time


In [1072]:
bowery.doors_time = time(bowery.doors_time)

In [1073]:
bowery.show_time = time(bowery.show_time)

In [1074]:
# change button from "Buy Now" to "On_Sale"
bowery.button[bowery.button == "Buy Tickets"] = "On_Sale"

In [1075]:
# clean adv price

def price(x):
    clean_price = []
    for i in x:
        if re.search("(Adv:) \$\d+\.\d+.\-.\$\d+\.\d+", i) is not None:
            m = re.search("(Adv:) \$\d+\.\d+.\-.\$\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        elif re.search("(Adv:) \$\d+\.\d+", i) is not None:
            m = re.search("(Adv:) \$\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        else:
            clean_price.append("NA")
    return clean_price

In [1076]:
bowery["adv_price"] = price(bowery.price)

In [1077]:
#clean door price

def door_price(x):
    clean_price = []
    for i in x:
        if re.search("(Door:) \$\d+\.\d+.\-.\$\d+\.\d+", i) is not None:
            m = re.search("(Door:) \$\d+\.\d+.\-.\$\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        elif re.search("(Door:) \$\d+\.\d+", i) is not None:
            m = re.search("(Door:) \$\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        else:
            clean_price.append("NA")
    return clean_price

In [1078]:
bowery["door_price"] = door_price(bowery.price)

In [1079]:
bowery = bowery.drop(['price'], 1)

In [1080]:
# drop rows where adv_price and door_price is NA
bowery = bowery[(bowery.adv_price != "NA") | (bowery.door_price != "NA")]

In [1081]:
# clean adv_price
def adv_price_split(x):
    clean_price = []
    for i in x:
        if re.search("\d+\.\d+ \- \$\d+\.\d+", i) is not None:
            m = re.search("\d+\.\d+ \- \$\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        elif re.search("\d+\.\d+", i) is not None:
            m = re.search("\d+\.\d+", i)
            clean_price.append(m.group(0))
            
        else:
            clean_price.append("NA")
    return clean_price

In [1082]:
bowery["adv_price"] = adv_price_split(bowery.adv_price)

In [1083]:
# split adv high price and low price
bowery['adv_price_low'], bowery['adv_price_high'] = bowery['adv_price'].str.split(' - ', 1).str

In [1084]:
bowery = bowery.drop(['adv_price'], 1)

In [1085]:
# remove dollar sign from adv_price_high
def rem_dollar(x):
    clean_dollar = []
    for i in x:
        try:
            m = re.search("\d+\.\d+", i)
            clean_dollar.append(m.group(0))
        except:
            clean_dollar.append("NA")
    return clean_dollar

In [1086]:
bowery.adv_price_high = rem_dollar(bowery.adv_price_high)

In [1087]:
def clean_door(x):
    clean_door = []
    for i in x:
        try:
            m = re.search("\d+\.\d+", i)
            clean_door.append(m.group(0))
        except:
            clean_door.append("NA")
    return clean_door

In [1088]:
bowery.door_price = clean_door(bowery.door_price)

In [1089]:
# prices to numeric
bowery.door_price = bowery.door_price.apply(pd.to_numeric, errors='coerce')
bowery.adv_price_high = bowery.adv_price_high.apply(pd.to_numeric, errors='coerce')
bowery.adv_price_low = bowery.adv_price_low.apply(pd.to_numeric, errors='coerce')

In [1090]:
# create price range column
bowery['adv_price_range'] = bowery.adv_price_high - bowery.adv_price_low

In [1091]:
# take only rows where adv price range is less than 5
bowery = bowery[(np.isnan(bowery.adv_price_range)) | (bowery.adv_price_range<5)]

In [1092]:
# add date time column - show time
bowery['date_time'] = bowery.date + " " + bowery.show_time

In [1093]:
from dateutil.parser import parse
def parse_time(x):
    empty_list = []
    for i in x:
        #parse(i)
        empty_list.append(parse(i))
    return empty_list
    

In [1094]:
# parse date_time column and date column
bowery.date_time = parse_time(bowery.date_time)
bowery.date = parse_time(bowery.date)

In [1095]:
group = bowery.groupby('button')['button']
group.size()

# drop where button == cancelled, coming soon, on sale soon, postponed

bowery = bowery[(bowery.button == 'On_Sale') | (bowery.button == 'Sold Out')]

In [1096]:
# create days_from_scrape_to_show column
bowery['scrape_day'] = parse("05/12/18")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1097]:
bowery['days_scrape_to_show'] = bowery.date - bowery.scrape_day

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1098]:
# make headline artists more searchable

def clean_headliner(x):
    clean_headliner= []
    for i in x:
        if re.search('\(', i) is not None:
            clean_headliner.append(re.sub('\(.*?\)', '', i).strip())
            
        elif re.search('(?<=: ).*$', i) is not None:
            m = re.search('(?<=: ).*$', i)
            clean_headliner.append(m.group(0))
            
        elif re.search('.+?(?= -)', i) is not None:
            m = re.search('.+?(?= -)', i)
            clean_headliner.append(m.group(0))
            
        else:
            clean_headliner.append(i)
    return clean_headliner

In [1099]:
bowery.headline_artist_name = clean_headliner(bowery.headline_artist_name)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [1100]:
# split headliners
bowery['headliner1'], bowery['headliner2'] = bowery['headline_artist_name'].str.split("[,\+/]+", 1).str


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1101]:
#drop festival of disruption
bowery = bowery[bowery.headliner1 != "Festival of Disruption"]

In [1102]:
# clean other artists
def clean_other_artists(x):
    clean_other_artists = []
    for i in x:
        if i == np.nan:
            clean_other_artists.append(i)
        
        elif re.search('(with special guests)', str(i)) is not None:
            m = re.search('(?<=(with )).*$', i)
            clean_other_artists.append(m.group(0))
            
        elif re.search('(with)', str(i)) is not None:
            m = re.search('(?<=(with )).*$', i)
            clean_other_artists.append(m.group(0))
            
        elif re.search('\(', str(i)) is not None:
            clean_other_artists.append(re.sub('\(.*?\)', '', i).strip())
        
        else:
            clean_other_artists.append(i)
    return clean_other_artists

In [1103]:
bowery.other_artists_name = clean_other_artists(bowery.other_artists_name)


In [1104]:
# split other artists
bowery['other_artists1'], bowery['other_artists2'] = bowery['other_artists_name'].str.split("[,\+/]+", 1).str

In [1105]:
# drop third or more other artists
def clean_other_artists2(x):
    clean_other_artists= []
    for i in x:
        if re.search(',', str(i)) is not None:
            clean_other_artists.append(re.sub('(?=,).*$', '', i).strip())
        else:
            clean_other_artists.append(i)
    return clean_other_artists
            

In [1106]:
bowery.other_artists2 = clean_other_artists2(bowery.other_artists2)

In [1107]:
headliners1 = bowery.headliner1

In [1108]:
def lstrip(x):
    empty_list = []
    for i in x:
        if type(i) == float:
            empty_list.append(i)
        else:
            #i.lstrip()
            empty_list.append(i.lstrip())
            
    return empty_list

In [1109]:
# strip extra starting spaces
bowery.headliner2 = lstrip(bowery.headliner2)
bowery.other_artists1 = lstrip(bowery.other_artists1)
bowery.other_artists2 = lstrip(bowery.other_artists2)

In [1110]:


headliners2 = bowery.headliner2
others1 = bowery.other_artists1
others2 = bowery.other_artists2

%store headliners2
%store others1
%store others2

Stored 'headliners2' (Series)
Stored 'others1' (Series)
Stored 'others2' (Series)


In [1111]:
%store -r headliners1_df
%store -r headliners2_df
%store -r others1_df
%store -r others2_df

In [1112]:
# rename artist_name
headliners2_df.rename(columns={'artist_name':'artist_name2'}, inplace=True)
others1_df.rename(columns={'artist_name':'other_artist_name1'}, inplace=True)
others2_df.rename(columns={'artist_name':'other_artist_name2'}, inplace=True)

In [1113]:
# rename other dup cols - artist pop
headliners2_df.rename(columns={'artist_popularity':'artist2_popularity'}, inplace=True)
others1_df.rename(columns={'artist_popularity':'artist_popularityO1'}, inplace=True)
others2_df.rename(columns={'artist_popularity':'artist_popularityO2'}, inplace=True)

In [1114]:
# rename other dup cols - artist fols
headliners2_df.rename(columns={'artist_followers':'artist2_followers'}, inplace=True)
others1_df.rename(columns={'artist_followers':'artist_followersO1'}, inplace=True)
others2_df.rename(columns={'artist_followers':'artist_followersO2'}, inplace=True)

In [1115]:
# rename other dup cols - artist gen
headliners2_df.rename(columns={'artist_genres':'artist2_genres'}, inplace=True)
others1_df.rename(columns={'artist_genres':'artist_genresO1'}, inplace=True)
others2_df.rename(columns={'artist_genres':'artist_genresO2'}, inplace=True)

In [1116]:
bowery = bowery.reset_index()
bowery = bowery.drop(['index'], 1)

In [1117]:
bowery = pd.concat([bowery, headliners1_df, headliners2_df, others1_df, others2_df], axis=1)

In [1118]:
# drop unessential cols and rows where artist_name is None - create boweryWorking
# drop doors_time, show_time, adv_price_range, headline_artist_name, other_artists_name
# drop date_time, scrape_day, headliner1, headliner2, other_artists1, other_artists2
# artist2_genres, artist_genresO1, artist_genresO2

In [1128]:
boweryWorking = bowery.drop(['doors_time', 'show_time', 'headline_artist_name', 
                             'other_artists_name','show_time', 'adv_price_range',
                             'date_time', 'scrape_day','headliner1','headliner2',
                             'other_artists1','other_artists2','artist2_genres','artist_genresO1','artist_genresO2'], axis=1, inplace=False)



In [1131]:
# drop rows where artist_name is null
boweryWorking = boweryWorking[boweryWorking.artist_name.notnull()]

In [1138]:
group = boweryWorking.groupby('venue')['venue']
showsPerVenue = group.size()

In [1157]:
# choose venues to scrape

#Notes:
# drop venues with less than 4 shows

# fete music hall combine ballroom and lounge
# royale to royale boston

# saturn is the one in birmingham, AL
# the national is in richmond, VA
# the sinclair is in cambridge, MA
showsPerVenue[showsPerVenue>3]

venue
Aisle 5                        5
Brooklyn Steel                43
Fête Music Hall - Ballroom     5
Fête Music Hall - Lounge       4
Gasa Gasa                     17
Great Scott                   50
Music Hall of Williamsburg    38
Rams Head Live!               22
Rough Trade NYC               44
Royale                        19
Saturn                        27
Starland Ballroom              6
Terminal 5                     7
Terminal West                  9
The EARL                       6
The National                  31
The NorVa                     28
The Sinclair                  55
Underground Arts              14
Variety Playhouse              7
Name: venue, dtype: int64

In [59]:
boweryWorking

,button,date,venue,city,state,week_day,door_price,adv_price_low,adv_price_high,days_scrape_to_show,...,artist_popularity,artist2_followers,artist_name2,artist2_popularity,artist_followersO1,other_artist_name1,artist_popularityO1,artist_followersO2,other_artist_name2,artist_popularityO2
0,On_Sale,2018-11-11,Gasa Gasa,New Orleans,LA,Sun,12.00,8.00,NaN,182 days,...,33.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
1,On_Sale,2018-11-12,Underground Arts,Philadelphia,PA,Mon,NaN,22.00,25.0,183 days,...,57.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
2,On_Sale,2018-11-14,The National,Richmond,VA,Wed,43.00,39.50,43.5,185 days,...,56.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
3,On_Sale,2018-11-15,The NorVa,Norfolk,VA,Thu,23.00,20.50,NaN,186 days,...,59.0,nan,None,NaN,47765,Nick Waterhouse,65.0,NaN,None,NaN
4,On_Sale,2018-11-23,Royale,Boston,MA,Fri,30.00,27.00,30.0,194 days,...,59.0,nan,None,NaN,47765,Nick Waterhouse,65.0,NaN,None,NaN
5,On_Sale,2018-11-30,Electric Factory,Philadelphia,PA,Fri,NaN,27.50,30.0,201 days,...,55.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
6,On_Sale,2018-12-08,Rough Trade NYC,Brooklyn,NY,Sat,18.00,15.00,NaN,209 days,...,58.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
7,On_Sale,2018-12-11,Great Scott,Allston,MA,Tue,12.00,12.00,NaN,212 days,...,38.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
8,On_Sale,2018-12-26,The National,Richmond,VA,Wed,23.00,20.00,NaN,227 days,...,6.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN
9,Sold Out,2018-12-27,Music Hall of Williamsburg,Brooklyn,NY,Thu,35.00,35.00,NaN,228 days,...,47.0,nan,None,NaN,nan,None,NaN,NaN,None,NaN


In [60]:
%store boweryWorking


Stored 'boweryWorking' (DataFrame)
